# Random search good parameters for models

In [ ]:
from players.atm import ATM
from players.ai_player import AIPlayer
from players.random_player import RandomPlayer
from training_env import TrainingEnv
from agents import build_dqn_agent, fit_agent, train_loop, load_agent_weights
from models import simple_model, complex_model, test_model
from util import visualize_history, use_jupyter, release_memory, print_stats, set_on_demand_memory_allocation
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

In [ ]:
from rl.memory import SequentialMemory

In [ ]:
set_on_demand_memory_allocation()

In [ ]:
use_jupyter()

In [ ]:
# How many players in table
NUMBER_OF_SEATS = 2
# Max betsize in simulation environment (shouldn't really matter with discrete relative to pot sizing)
MAX_BET = 100000
# 'norm' (normalized) or 'one-hot' < how to encode player hand ranking from 7642 unique values
RANK_ENCODING = 'norm'

WINDOW = 20
MODEL = complex_model
OPPONENT = RandomPlayer()

STEPS = 2000

LEARNING_PARAMS_ITERATIONS = 5
BATCHING_PARAMS_ITERATIONS = 5

In [ ]:
all_params = {
    'limit': [128, 256, 1024, 4096, 32768],
    'target_model_update': [0.00001, 0.0001, 0.001, 0.01, 0.1, 0.2, 10, 100, 1000, 10000],
    'batch_sz': [128], #[8, 16, 32, 64],
    'train_interval': [64], #[1, 10, 100, 1000],
    'gamma': [.9, .99, .999, .9999],
    'lr': [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
    'beta_1': [.9, .99, .999],
    'beta_2': [.9, .99, .999, .9999, .99999]
}
tried_params = []

In [ ]:
env = TrainingEnv.build_environment(OPPONENT, NUMBER_OF_SEATS, debug=False)

In [ ]:
def test_params(params, env, steps):
    
    # target_model_update - how often to update target model (or if in 0...1 range, the soft update weight)
    # window_length - how many timesteps to look into past (will multiply observation space by this, be careful)
    # enable_double_dqn - https://arxiv.org/pdf/1509.06461.pdf
    # enable_dueling_network - ???
    # train_interval - every how many steps to run a train cycle 
    # n_warmup_steps - how many steps to run without training
    # batch_size - number of (s, a, G) triplets to train on in one training cycle (as a batch)
    # gamma - future reward discount essentially
    # memory_interval - how often to add last step to memory buffer (discarding every other)
    
    # lr - learning rate
    # beta_1 - L1 normalization
    # beta_2 - L2 normalization
    # epsilon - Fuzz factor
    # decay - Learning rate decay

    warmup_steps = max(params['train_interval'], params['batch_sz']) + 1
    
    model = MODEL(WINDOW, env.n_observation_dimensions, env.n_actions)
    
    memory = SequentialMemory(limit=int(params['limit']), window_length=WINDOW)
    optimizer = Adam(lr=params['lr'], beta_1=params['beta_1'], beta_2=params['beta_2'], epsilon=None, 
                     decay=0.0, amsgrad=True)
    agent = build_dqn_agent(model, env.n_actions, memory=memory, window_length=WINDOW, 
                            target_model_update=params['target_model_update'], 
                            enable_double_dqn=True, enable_dueling_network=True, dueling_type='avg', 
                            train_interval=params['train_interval'], n_warmup_steps=warmup_steps, 
                            batch_size=params['batch_sz'], gamma=params['gamma'], memory_interval=1, 
                            optimizer=optimizer)
    agent, hist = fit_agent(agent, env, steps, start_from_scratch=True, verbose=1, debug=False)
    print_stats(hist)
    release_memory([agent])
    return sum(hist.history['episode_reward'])

In [ ]:
def get_params(params=None, chance=.3):
    if params is None:
        params = {}
        chance = 1
    attempts = 0
    while True:
        for parameter, value_options in all_params.items():
            if np.random.random() < chance:
                params[parameter] = np.random.choice(value_options)
        values = list(params.values())
        if not values in tried_params:
            tried_params.append(values)
            break
        attempts += 1
        if attempts > 100:
            return False     
    return params

In [ ]:
best_params = None
best_reward = -float('inf')
for i in range(LEARNING_PARAMS_ITERATIONS):
    print('\nITERATION %s' % str(i))
    new_params = get_params(params=best_params)
    if not new_params:
        break
    reward = test_params(new_params, env, STEPS)
    if reward > best_reward:
        best_reward = reward
        best_params = new_params
        print('New best params:', best_params)

In [ ]:
print(best_params)
# {'limit': 200, 'target_model_update': 1000.0, 'batch_sz': 128, 'train_interval': 50, 
# 'gamma': 0.9999, 'lr': 1e-05, 'beta_1': 0.999, 'beta_2': 0.9}

In [ ]:
all_params = {
    'limit': [best_params['limit']],
    'target_model_update': [best_params['target_model_update']],
    'batch_sz': [8, 16, 32, 64, 128],
    'train_interval': [1, 10, 100, 1000],
    'gamma': [best_params['gamma']],
    'lr': [best_params['lr']],
    'beta_1': [best_params['beta_1']],
    'beta_2': [best_params['beta_2']]
}
tried_params = []

In [ ]:
import time

best_params = None
best_reward_rate = -float('inf')
for i in range(BATCHING_PARAMS_ITERATIONS):
    print('\nITERATION %s' % str(i))
    new_params = get_params(params=best_params)
    if not new_params:
        break
    start_time = time.time()
    reward = test_params(new_params, env, STEPS)
    time_taken = time.time() - start_time
    reward_rate = reward / time_taken
    if reward_rate > best_reward_rate:
        best_reward_rate = reward_rate
        best_params = new_params
        print('New best params:', best_params)